## Install necessary packages

In [1]:
!pip install lang chain

  Preparing metadata (setup.py) ... done
  Created wheel for chain: filename=chain-1.0-py3-none-any.whl size=3501 sha256=58bfd0b442ca91bdeca3acc89eeb1b7b4be7f004acf0ab97c7d26c22ee9a1559
  Stored in directory: /Users/krystalli/Library/Caches/pip/wheels/40/b3/8d/558feac3d2291ca93f8b7806732526b2176e27db163741fc6f
Successfully built chain


In [3]:
!pip install langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 8.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.5/266.5 kB 26.2 MB/s eta 0:00:00


## Test if the LLM works

In [7]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0,
    groq_api_key = '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',
)

res = llm.invoke("The city that has the best techno club is?")
print(res.content)

What a subjective question! The answer, of course, depends on personal taste, musical preferences, and cultural context. That being said, here are some of the most renowned techno clubs in the world, often cited as some of the best:

1. **Berghain (Berlin, Germany)**: Known for its strict door policy, Berghain is a temple of techno and one of the most iconic clubs in the world. Its reputation is built on its 24-hour parties, showcasing the best in techno and minimal techno.
2. **Tresor (Berlin, Germany)**: Another legendary club in Berlin, Tresor has been a hub for techno and electronica since the 1990s. Its small, intimate space creates an intense atmosphere, perfect for head-banging to heavy techno beats.
3. **fabric (London, UK)**: This London institution is a powerhouse of techno, hosting some of the biggest names in the scene. Its cutting-edge sound system and top-notch booking policy make it a must-visit for techno enthusiasts.
4. **Amnesia (Ibiza, Spain)**: This iconic club on t

## Extract text from Nike job posting website using lang chain WebBaseLoader

In [17]:
!pip install -qU langchain_community beautifulsoup4

In [24]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-39879?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Machine Learning Engineer - Supply Chain

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsMachine Learning Engineer - Supply ChainBeaverton, OregonBecome a Part

#### Now we only extract the full text, but there are unnecessary info, and is not structured.

## Use PromptTemplate to extract only useful info

In [42]:
from langchain_core.prompts import PromptTemplate

# prompt for extracting useful info
prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them as a JSON object with the 
        following keys: `role`, `experience`, `skills`, and `description`.
        Each key should have a value that corresponds to the details of the job posting.
        Only return a single, valid JSON object without any additional text or preamble.
        ### VALID JSON OBJECT:
        """
)

# get the chain
chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)
type(res.content)

Here is the extracted job posting in the specified JSON format:

```
{
    "role": "Machine Learning Engineer - Supply Chain",
    "experience": "1-3 years of professional experience in software engineering, data engineering, machine learning, or related field",
    "skills": ["Python", "algorithm and data structures", "AWS", "database technology (e.g. Postgres, Redis)", "data processing technology (e.g. EMR)", "Docker", "CI/CD", "Spark", "Kubernetes", "Jenkins", "Databricks", "Terraform"],
    "description": "Develop robust advanced analytics and machine learning solutions that have a direct impact on the Supply Chain infrastructure. As a Machine Learning Engineer, you will own projects end-to-end - from conception to operationalization, demonstrating an understanding of the full software development lifecycle."
}
```


str

#### Now we have the useful info! But it's in strings, need to get it in a JSON format

## Use JsonOutputParser to put the info into JSON format

In [44]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Machine Learning Engineer - Supply Chain',
 'experience': '1-3 years of professional experience in software engineering, data engineering, machine learning, or related field',
 'skills': ['Python',
  'algorithm and data structures',
  'AWS',
  'database technology (e.g. Postgres, Redis)',
  'data processing technology (e.g. EMR)',
  'Docker',
  'CI/CD',
  'Spark',
  'Kubernetes',
  'Jenkins',
  'Databricks',
  'Terraform'],
 'description': 'Develop robust advanced analytics and machine learning solutions that have a direct impact on the Supply Chain infrastructure. As a Machine Learning Engineer, you will own projects end-to-end - from conception to operationalization, demonstrating an understanding of the full software development lifecycle.'}

In [46]:
type(json_res)

dict

## Prepare ChromaDB

In [51]:
import pandas as pd

df = pd.read_csv("my_portfolio_example.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [53]:
import uuid
import chromadb

# Use PersistentClient() instead of Client() to ensure that data is saved to disk and available across sessions.
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

# when the DB is created for the first time (aka empty), iterrate through the df to store the date into db. 
# metadata = portfolio links, documents = techstack
if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [61]:
# test with queries to see if db returns the most relevant portfolio link
links = collection.query(query_texts=['Experience in Python', 'Expertise in React Native'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [86]:
# check out the skills from the scrapped text
job = json_res
job['skills']

['Python',
 'algorithm and data structures',
 'AWS',
 'database technology (e.g. Postgres, Redis)',
 'data processing technology (e.g. EMR)',
 'Docker',
 'CI/CD',
 'Spark',
 'Kubernetes',
 'Jenkins',
 'Databricks',
 'Terraform']

In [88]:
job

{'role': 'Machine Learning Engineer - Supply Chain',
 'experience': '1-3 years of professional experience in software engineering, data engineering, machine learning, or related field',
 'skills': ['Python',
  'algorithm and data structures',
  'AWS',
  'database technology (e.g. Postgres, Redis)',
  'data processing technology (e.g. EMR)',
  'Docker',
  'CI/CD',
  'Spark',
  'Kubernetes',
  'Jenkins',
  'Databricks',
  'Terraform'],
 'description': 'Develop robust advanced analytics and machine learning solutions that have a direct impact on the Supply Chain infrastructure. As a Machine Learning Engineer, you will own projects end-to-end - from conception to operationalization, demonstrating an understanding of the full software development lifecycle.'}

In [71]:
# replace query with the scraped skills from job posting
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}

#### It returns the most relevant portfolios!

## Generate the cold email

In [94]:
# prompt for the email
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Krystal, a business development executive at Kiki.ai. Kiki.ai is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Kiki.ai 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Kiki.ai's portfolio: {link_list}
        Remember you are Krystal, BDE at Kiki.ai. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

# get the chain
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock the Power of AI in Supply Chain Management

Dear [Hiring Manager],

I came across the job description for a Machine Learning Engineer - Supply Chain at [Company Name] and was impressed by the opportunity to develop advanced analytics and machine learning solutions that can have a direct impact on the Supply Chain infrastructure.

At Kiki.ai, we specialize in facilitating the seamless integration of business processes through automated tools, empowering enterprises to achieve scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has extensive experience in developing innovative solutions that cater to the unique needs of our clients.

I'd like to highlight that our portfolio includes projects that showcase our expertise in machine learning, data engineering, and software development. For instance, you can explore our Python portfolio at [https://example.com/python-portfolio] and our machine learning portfolio at [https: